In [13]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
results = requests.get(url).text
soup = BeautifulSoup(results, 'html.parser')
page = soup.find_all("tr")
df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])
i = 0
for row in page:
    a = row.get_text().split('\n')[1:4]
    if (len(a) == 3 and i > 0 and i < 291 and a[1] != 'Not assigned'):
        if a[2] == 'Not assigned':
            if a[0] in df.PostalCode.values:
                index_val = df.loc[df['PostalCode'] == a[0]].index[0]
                string_val = df.iloc[index_val].Neighborhood + ", " + a[1]
                new_df = pd.DataFrame({'Neighborhood': [string_val]}, index=[int(index_val), 1])
                df.update(new_df)
            else:
                df = df.append({'PostalCode': a[0], 'Borough': a[1], 'Neighborhood': a[1]}, ignore_index=True)
        else:
            if a[0] in df.PostalCode.values:
                index_val = df.loc[df['PostalCode'] == a[0]].index[0]
                string_val = df.iloc[index_val].Neighborhood + ", " + a[2]
                new_df = pd.DataFrame({'Neighborhood': [string_val]}, index=[int(index_val), 1])
                df.update(new_df)
            else:
                df = df.append({'PostalCode': a[0], 'Borough': a[1], 'Neighborhood': a[2]}, ignore_index=True)
    i +=1

df2 = pd.read_csv('http://cocl.us/Geospatial_data')
df = df.merge(df2, left_on='PostalCode', right_on='Postal Code', how='inner')
df = df.drop(['Postal Code'], axis=1)






In [54]:
print(df.shape)

(103, 3)


In [14]:
df = df[df['Borough'].str.contains("Toronto")]
df

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [10]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [15]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto